# 0. Preparación del entorno

## Con el siguiente bloque de código importamos nuestras bibliotecas
### Estas bibliotecas nos permiten simplificar el código empleado para que con pocas líneas podamos construir un programa que extraiga las regiones potencialmente codificantes de un genoma procariótico

In [ ]:
import pandas as pd
import numpy  as np
import re
import credentials
from Bio import Entrez

### Si deseas instalar estas bibliotecas en tu entorno, puedes ejecutar los siguientes comandos:

```bash
pip3 install --upgrade pandas
pip3 install --upgrade numpy
pip3 install --upgrade re
pip3 install --upgrade Biopython
```

# 1. Funciones para construir un buscador de ORFs con python

## 1.1. Primeramente debemos definir el código genético estándar como un diccionario de python
### La siguiente estructura de datos te permite cambiar a otros códigos genéticos como por ejemplo de las mitocondrias que pueden usar el codón `TGA` para decodificar triptofano en vez de un codón de paro

In [ ]:
genetic_code = {"AAA":"K","AAC":"N","AAG":"K","AAT":"N","ACA":"T","ACC":"T","ACG":"T","ACT":"T",
                "AGA":"R","AGC":"S","AGG":"R","AGT":"S","ATA":"I","ATC":"I","ATG":"M","ATT":"I",
                "CAA":"Q","CAC":"H","CAG":"Q","CAT":"H","CCA":"P","CCC":"P","CCG":"P","CCT":"P",
                "CGA":"R","CGC":"R","CGG":"R","CGT":"R","CTA":"L","CTC":"L","CTG":"L","CTT":"L",
                "GAA":"E","GAC":"D","GAG":"E","GAT":"D","GCA":"A","GCC":"A","GCG":"A","GCT":"A",
                "GGA":"G","GGC":"G","GGG":"G","GGT":"G","GTA":"V","GTC":"V","GTG":"V","GTT":"V",
                "TAA":"*","TAC":"Y","TAG":"*","TAT":"Y","TCA":"S","TCC":"S","TCG":"S","TCT":"S",
                "TGA":"*","TGC":"C","TGG":"W","TGT":"C","TTA":"L","TTC":"F","TTG":"L","TTT":"F"}

## 1.2. Con el siguiente bloque de código onstruimos una función sumamente simple para poner una secuencia en reverso complementario

1. Primero ponemos todos los nucleótidos en máyusculas
2. Luego remplazamos las bases por su reverso complementario, pero con minúsculas
3. Luego regresamos la secuencia a mayúsculas
4. Finalmente ponemos la secuencia en reverso con la notación `[::-1]`

In [ ]:
def reverse_complement(sequence):
    sequence = sequence.upper().replace("A","t").replace("C","g").replace("G","c").replace("T","a").upper()[::-1]
    return sequence

## 1.3. La siguiente celda contiene una función para partir nuestra secuencia en codones
### Avanzamos de 3 en 3 nucleótidos y agregamos cada triplete a una lista de python

In [ ]:
def get_codon_list(genome):
    codon_list = []
    for i in range(0, len(genome), 3):
        codon = genome[i:i+3]
        codon_list.append(codon)
    return codon_list

## 1.4. En la siguiente celda, tenemos una función nos permite obtener el codón de inicio más proximo a un codón de paro
### En caso de que un codón de paro no tenga un codón de inicio aledaño, nos regresa un valor nulo

In [ ]:
def get_nearest_start(stop_pos_df,start_pos_df,stop_pos):
    cur_stop_pos_index = stop_pos_df[stop_pos_df["stop_pos"]==stop_pos].index[0]
    if(cur_stop_pos_index==0):
        prev_stop_pos = 0
    elif(cur_stop_pos_index<len(stop_pos_df)):
        prev_stop_pos_index = cur_stop_pos_index - 1
        prev_stop_pos = stop_pos_df["stop_pos"][prev_stop_pos_index]
    cur_start_pos_index_list = start_pos_df[(start_pos_df["start_pos"]>=prev_stop_pos) & (start_pos_df["start_pos"]<stop_pos)]["start_pos"].index
    if (len(cur_start_pos_index_list)==0):
        cur_start_pos = np.nan
    else:
        cur_start_pos_index = cur_start_pos_index_list[0]
        cur_start_pos = start_pos_df["start_pos"][cur_start_pos_index]
    return cur_start_pos

## Breve repaso para traducir una secuencia nucleotídica

### Consideremos la siguiente secuencia de 21 nucleótidos
### `ATGAAACGTATTGTACCCTGA`

### Si deseamos traducir dicha secuencia en aminoácidos, podemos hacerlo en seis marcos de lectura

|Marco +1                                 |Marco +2                                |Marco +2                               |
|:---------------------------------------:|:--------------------------------------:|:-------------------------------------:|
|`ATG - AAA - CGT - ATT - GTA - CCC - TGA`|`TGA - AAC - GTA - TTG - TAC - CCT - GA`|`GAA - ACG - TAT - TGT - ACC - CTG - A`|
|` M     K     R     I     V     P     *` |` *     N     V     L     Y     P     X`|` E     T     Y     C     T     L    X`|

|Marco -1                                 |Marco -2                                |Marco -2                               |
|:---------------------------------------:|:--------------------------------------:|:-------------------------------------:|
|`TCA - GGG - TAC - AAT - ACG - TTT - CAT`|`CAG - GGT - ACA - ATA - CGT - TTC - AT`|`AGG - GTA - CAA - TAC - GTT - TCA - T`|
|` S     G     Y     N     T     F     H` |` Q     G     T     I     R     F     X`|` R     V     Q     Y     V     S    X`|

## 1.5. Con el siguiente bloque de código construimos una función para obtener las posiciones de los distintos ORFs en los 3 distintos marcos de lectura

### Esta función toma 3 argumentos:
* Una secuencia genómica
* Un marco de traducción (1,2,3,-1,-2,-3)
* Un umbral para excluir ORFs de tamaño pequeño

### Nuestra función realiza los siguientes pasos:

1. Primero calcula la longitud del genoma
2. Construye una lista de codones a partir del marco de lectura especificado
3. La búsqueda de ORFs se realiza a partir de los codones de paro, y la función que construimos anteriormente nos permite encontrar cual es el codon de inicio que le corresponde a cada codon de paro (en caso de que encuentre uno)
4. Posteriormente calcula la longitud del ORF y excluye los ORFs que sean de un tamaño inferior al umbral específicado

### La función nos entrega una lista de posiciones que corresponden al inicio y término del ORF en cada marco de lectura

In [ ]:
def get_orf_pos_list(genome,frame,orf_size):
    genome_len = len(genome)
    if frame<0:
        genome = reverse_complement(genome)
        tmp_frame = -1 * frame
        genome = genome[tmp_frame-1:]
    else:
        genome = genome.upper()
        genome = genome[frame-1:]
    codon_list     = get_codon_list(genome)
    stop_pos_list  = []
    orf_pos_list   = []
    codon_df       = pd.DataFrame(codon_list,columns=["codon"])
    codon_df       = pd.DataFrame(codon_list,columns=["codon"])
    start_pos_list = list(codon_df[codon_df["codon"]=="ATG"].index)
    stop_pos_list  = list(codon_df[(codon_df["codon"]=="TAA") | (codon_df["codon"]=="TAG") | (codon_df["codon"]=="TGA")].index)
    start_pos_df   = pd.DataFrame(start_pos_list,columns=["start_pos"])
    stop_pos_df    = pd.DataFrame(stop_pos_list,columns=["stop_pos"])
    pos_df         = stop_pos_df.copy()
    pos_df["start_pos"] = pos_df["stop_pos"].apply(lambda x: get_nearest_start(stop_pos_df,start_pos_df,x))
    pos_df = pos_df[pos_df["start_pos"]>0]
    pos_df["orf_len"]   = pos_df["stop_pos"] - pos_df["start_pos"] + 1
    pos_df = pos_df[pos_df["orf_len"]>=orf_size]
    if frame >0:
        pos_df["orf_start_pos"] = ((pos_df["start_pos"] * 3) + 1) + (frame - 1)
        pos_df["orf_stop_pos"]  = ((pos_df["stop_pos"]  + 1) * 3) + (frame - 1)
    elif frame<0:
        pos_df["orf_start_pos"] = (genome_len + (1+frame) ) - ( pos_df["start_pos"]     * 3)
        pos_df["orf_stop_pos"]  = (genome_len + (1+frame) ) - ((pos_df["stop_pos"] + 1) * 3)
    orf_pos_list = pos_df[["orf_start_pos","orf_stop_pos"]].astype(int).values.tolist()
    return orf_pos_list

## 1.6. Con la siguiente función podemos obtener la secuencia nucleotídica de cada ORF tomando sus coordenadas como indices de python para obtener sub-secuencias a partir de una cadena de texto (el genoma)

### La función toma 3 argumentos
* Un dataframe de pandas
* Una secuencia genómica
* El identificador de un ORF

### A partir del identificador del ORF, la función determina la cadena en la que se encuentra así como las coordenadas de inicio y de término

### A partir de esta información la función obtiene la secuencia, ya sea en sentido positivo (`strand = "+"`) o en reverso complementario (`strand = "-"`)

In [ ]:
def get_orf_sequence(orf_df,genome,orf_id):
    strand    = list(orf_df[orf_df["id"]==orf_id]["strand"])[0]
    start_pos = int(orf_df[orf_df["id"]==orf_id]["start"])
    stop_pos  = int(orf_df[orf_df["id"]==orf_id]["stop"])    
    if(strand=="-"):
        orf_sequence = genome[start_pos:stop_pos]
        orf_sequence = reverse_complement(orf_sequence)
    else:
        orf_sequence = genome[start_pos-1:stop_pos]
    return orf_sequence

## 1.7. Finalmente, con el siguiente bloque de código construimos una función para traducir ORFs

### Recuerdas el diccionario con el código genético que definimos anteriormente?
### Recuerdas la función que construimos para transformar nuestras secuencias en listas de codones?
### La función realiza los siguientes pasos:
* Toma la secuencia del ORF, que obtuvimos con `get_orf_sequence()`
* La convierte en una lista de codones con `get_codon_list()`
* Mapea cada codón al diccionario del código genético para determinar que amino ácido le corresponde a cada codón

In [ ]:
def translate_orf(orf_sequence):
    amino_acid_str = ""
    codon_list = get_codon_list(orf_sequence)
    for codon in codon_list:
        amino_acid = genetic_code[codon]
        amino_acid_str += amino_acid
    return amino_acid_str

# 2. Manos a la obra

## 2.1. Vamos a descargar un genoma usando la biblioteca de NCBI - Entrez

### Con las siguientes líneas de código puedes descargar secuencias en formato `fasta`
### La última línea elimina el encabezado de la secuencia y nos deja el genoma en una sola línea de texto

In [ ]:
genome_name = "NC_041953"
#genome_name = "JAFELC02"
genome = Entrez.efetch(db="nucleotide", id=genome_name, rettype="fasta", retmode="text").read()
genome = re.sub(r"\>.*\n", "", genome).replace("\n","")

## 2.2. Posteriormente obtenemos las posiciones de los ORFs mayores a 100 amino ácidos en cada uno de los seis marcos de lectura usando las funciones que definimos en el paso 1

In [ ]:
plus_one_pos_list    = get_orf_pos_list(genome, 1,100)
plus_two_pos_list    = get_orf_pos_list(genome, 2,100)
plus_three_pos_list  = get_orf_pos_list(genome, 3,100)
minus_one_pos_list   = get_orf_pos_list(genome,-1,100)
minus_two_pos_list   = get_orf_pos_list(genome,-2,100)
minus_three_pos_list = get_orf_pos_list(genome,-3,100)

## 2.3. A partir de estas listas de coordenadas, construimos dos dataframes:

* Los ORFs localizados en la cadena positiva del genoma
* Los ORFs localizados en la cadena negativa del genoma

In [ ]:
plus_orf_pos_list  = plus_one_pos_list  + plus_two_pos_list  + plus_three_pos_list
minus_orf_pos_list = minus_one_pos_list + minus_two_pos_list + minus_three_pos_list
plus_orf_df = pd.DataFrame(plus_orf_pos_list,columns=["start","stop"])
minus_orf_df = pd.DataFrame(minus_orf_pos_list,columns=["stop","start"])

## 2.4. Luego agregamos la información de la cadena a cada uno de los dataframes
### Habiendo etiquetado nuestros ORFs, podemos concatenarlos en un sólo objeto

In [ ]:
plus_orf_df["strand"]  = "+"
minus_orf_df["strand"] = "-"
plus_orf_df = plus_orf_df[["start","stop","strand"]]
minus_orf_df = minus_orf_df[["start","stop","strand"]]
orf_df = pd.concat([plus_orf_df,minus_orf_df],ignore_index=True)

## 2.4.1. Al concatenar los dataframes, los ORFs no quedan ordenados de forma lógica, por lo que debemos ordenarlos con base en su coordenada de inicio
### Al tener nuestros ORFs ordenados, podemos agregarles una etiqueta que indique de qué genoma vienen y cual es su número de aparición en el genoma

In [ ]:
orf_df = orf_df.sort_values(by=["start"],ignore_index=True)
orf_df["genome"] = genome_name
orf_df["id"] = np.arange(len(orf_df)).astype(str)
orf_df["id"] = orf_df["genome"]+"."+orf_df["id"]

## 2.5. Ya con los ORFs identificados, podemos obtener su secuencia codificante y agregarla como una columna extra en el dataframe

In [ ]:
orf_df["orf_sequence"] = orf_df["id"].apply(lambda x: get_orf_sequence(orf_df,genome,x))

## 2.6. Y podemos agregar también la traducción de dicha secuencia usando `translate_orf()`

In [ ]:
orf_df["orf_translation"] = orf_df["orf_sequence"].apply(lambda x: translate_orf(x))

### Nuestra predicción de ORFs va acumulando información que no solamente es parecida a la estructura del formato de anotaciones genómicas `General Feature Format`, sino que también es compatible con dicho formato.
#### A partir de nuestro dataframe podemos construir un archivo `.gff` muy parecido a los que están depositados en GenBank
### Para ello agregaremos una columna con la información del ORF

In [ ]:
orf_df["info"] = "Id="+orf_df["id"]+";translation="+orf_df["orf_translation"]+";product=hypothetical protein"

## 2.7. Agregamos un par de columnas que indican que tipo de información y de que fuente estamos obteniendo dicha información

In [ ]:
orf_df["score"]  = "."
orf_df["phase"]  = "."
orf_df["type"]   = "CDS"
orf_df["source"] = "python_orfinder"

## 2.8. Guardamos nuestro dataframe en un archivo `.gff`
### Con la función `.to_csv()` de pandas, podemos almacenar la información de nuestro dataframe a un archivo tabular, en este punto ya tenemos la información necesaria para guardar nuestras predicciones en un archivo `.gff`

In [ ]:
orf_df[["genome","source","type","start","stop","score","strand","phase","info"]].to_csv(genome_name+".gff",sep="\t",header=False,index=False)

## 2.9. Guardamos las traducciones de nuestros ORFs en formato `.fasta` para análisis subsecuentes

In [ ]:
with open(genome_name+".faa","w") as file_name:
    fasta_str = ""
    for orf_id,orf_translation in list(orf_df[["id","orf_translation"]].values):
        fasta_str += ">" + orf_id + "\n" + orf_translation +"\n"
    file_name.write(fasta_str)